# Како се мењао еколошки отисак у последњих пола века?
У скупу података који користимо налази се и табела која приказује промене вредности глобалног еколошког отиска у времену. Учитаћемо табелу *NFA 1961-2017* и анализирати доступне податке. Одмах ћемо променити називе варијабли да не бисмо накнадно морали да мењамо ознаке оса и легенди.

In [1]:
import pandas as pd

otisakTrend = pd.read_csv('podaci/NFA 1961-2017.csv')

# Оса (axis) 1 садржи називе колона које мењамо у самом скупу оператором inplace=True
otisakTrend.set_axis(['Година', 'Оранице', 'Пашњаци', 'Шуме', 'CO2', 'Риба', 'Земљиште', 'Еколошки отисак', 'Биокапацитет'], axis=1, inplace=True)

# Приказујемо основне информације о скупу
otisakTrend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Година           57 non-null     int64  
 1   Оранице          57 non-null     float64
 2   Пашњаци          57 non-null     float64
 3   Шуме             57 non-null     float64
 4   CO2              57 non-null     float64
 5   Риба             57 non-null     float64
 6   Земљиште         57 non-null     float64
 7   Еколошки отисак  57 non-null     float64
 8   Биокапацитет     57 non-null     float64
dtypes: float64(8), int64(1)
memory usage: 4.1 KB


Погледаћемо најпре како се мењао однос вредности укупног еколошког отиска и расположивих биокапацитета у периоду од 1961. до 2017. године.

In [2]:
import plotly.express as px

# Параметар markers је доступан само у новијим верзијама библиотеке plotly,
# па ће можда бити потребно да је освежите користећи команду pip install --upgrade plotly
graf = px.line(otisakTrend, x='Година', y=['Еколошки отисак', 'Биокапацитет'], 
               markers=True, width=800, height=400, template='plotly_white')

# Мало подешавамо изглед линија, тачкица, наслове...
graf.update_layout(margin=dict(l=80, r=30, t=60, b=0), 
                   xaxis_title='', yaxis_title='Глобални хектари по становнику',
                   legend=dict(title='', y=0.01, x=0.01, bgcolor='rgba(0,0,0,0)', font=dict(size=12)),
                   title=dict(text='Промене глобалног еколошког отиска и биокапацитета у периоду 1961-2017.'), 
                              title_xanchor='center', font=dict(size=11), title_x=0.5
                  )

# Повећавамо број подеока на x-оси
graf.update_xaxes(nticks=13) 

# Приказујемо графикон
graf.show()

Разлике у правцима линија показују да од 1970-тих година однос између укупног еколошког отиска и биокапацитета Земље добија негативан тренд. То се може приписати не толико порасту потреба за природним ресурсима, колико њиховом трошењу или уништавању. С обзиром на то да се биокапацитет планете Земље изражава у глобалним хектарима по становнику, на овај драстичан пад вероватно је утицао и глобални пораст популације. Уочите да су се одређени глобални феномени, као што су рецесије __[1973-1975.](https://en.wikipedia.org/wiki/1973%E2%80%931975_recession)__ и __[2007-2009](https://en.wikipedia.org/wiki/Great_Recession)__, одразили и на смањење еколошког отиска, првенствено због смањених економских активности, али и платежне моћи становника.
<BR><BR>
У наредном кораку ћемо погледати како су се мењале вредности различитих компонената еколошког отиска у периоду од 1961. до 2017. Овога пута смо скупу додали и варијаблу `Земљиште` (енгл. *build-up land*) која се односи на потребе за грађевинским земљиштем, индустријским земљиштем и површинама за рударске копове. Промене ћемо приказати помоћу *дијаграма попуњених површина* (енгл. *filled area*). Међутим, пре тога морамо да трансформишемо табелу у нешто другачији формат. Погледајмо најпре како изгледа оригинална табела.

In [3]:
otisakTrend.head(5)

,Година,Оранице,Пашњаци,Шуме,CO2,Риба,Земљиште,Еколошки отисак,Биокапацитет
0,1961,0.146385,0.085419,0.137855,0.321651,0.030571,0.008470,2.27,3.11
1,1962,0.150852,0.085740,0.138346,0.334041,0.031811,0.008872,2.30,3.07
2,1963,0.152748,0.087373,0.139699,0.357652,0.032797,0.009107,2.35,3.01
3,1964,0.156199,0.086305,0.144157,0.379609,0.032011,0.009480,2.39,2.96
4,1965,0.158445,0.088201,0.145523,0.399297,0.034353,0.009770,2.43,2.91


Видимо да табела има 9 колона, од којих нам је заправо потребно само првих седам које се односе на компоненте еколошког отиска и годину. При томе не рачунамо крајњу леву колону која служи као нека врста идентификације записа и назива се *индексом*. У наредном кораку ћемо издвојити поменутих седам колона, а варијабли *Година* доделити статус индекса.

In [4]:
otisakTrendIndeks = otisakTrend.iloc[:,0:7].set_index('Година') # У наставку се могу низати функције
otisakTrendIndeks.head(5)

,Оранице,Пашњаци,Шуме,CO2,Риба,Земљиште
Година,,,,,,
1961,0.146385,0.085419,0.137855,0.321651,0.030571,0.008470
1962,0.150852,0.085740,0.138346,0.334041,0.031811,0.008872
1963,0.152748,0.087373,0.139699,0.357652,0.032797,0.009107
1964,0.156199,0.086305,0.144157,0.379609,0.032011,0.009480
1965,0.158445,0.088201,0.145523,0.399297,0.034353,0.009770


Колоне смо одабрали командом __[`iloc`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html)__ којом се на основу целобројних вредности (енгл. *integer-locaton*) издвајају жељене ћелије табеле са подацима. Први аргумент у угластим заградама односи се на редове, а други на колоне. У нашем примеру издвојили смо све редове пошто смо ознаку `:` унели без распона бројева. Обратите пажњу да се колоне нумеришу бројевима од *0*  до *број колона - 1*, као и да колона са редним бројем 7 није издвојена. То значи да смо командом `iloc[:,0:7]` издвојили све записе из првих седам колона. Поред тога, колону *Година* смо претворили у индекс, што можете да видите по томе што је назив колоне издвојен у засебан ред. На крају ћемо „препаковати“ податке користећи функцију __[`stack`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html)__ тако да се колоне са називима варијабли за сваку годину претворе у редове. На тај начин се добија табела са тзв. *хијерархијским индексима* два нивоа.  

In [5]:
otisakTrendStack = otisakTrendIndeks.stack()
otisakTrendStack.head(10)

Година          
1961    Оранице     0.146385
        Пашњаци     0.085419
        Шуме        0.137855
        CO2         0.321651
        Риба        0.030571
        Земљиште    0.008470
1962    Оранице     0.150852
        Пашњаци     0.085740
        Шуме        0.138346
        CO2         0.334041
dtype: float64

На крају остаје само да ресетујемо индексе и претворимо их у класичне колоне табеле. Бивши индекси који нису имали називе биће претворени у колоне под називима *level_1*, *level_2* итд., док ће колонама бити додељени називи *0*, *1*, *2* итд. 

In [6]:
otisakTrendPlot = otisakTrendStack.reset_index()
otisakTrendPlot.head(5)

,Година,level_1,0
0,1961,Оранице,0.146385
1,1961,Пашњаци,0.085419
2,1961,Шуме,0.137855
3,1961,CO2,0.321651
4,1961,Риба,0.030571


Зато ћемо приликом ресетовања индекса одмах променити и називе колона.  

In [7]:
otisakTrendPlot = otisakTrendStack.reset_index().rename(columns = {'level_1': 'Тип', 0: 'Вредност у gha'})
otisakTrendPlot.head(5)

,Година,Тип,Вредност у gha
0,1961,Оранице,0.146385
1,1961,Пашњаци,0.085419
2,1961,Шуме,0.137855
3,1961,CO2,0.321651
4,1961,Риба,0.030571


Табела је коначно спремна и можемо да исцртамо дијаграм попуњених површина. Имајте на уму да смо трансформације обавили у неколико корака због бољег разумевања и да сте све функције (`stack()`, `reset_index()` и `rename()`) могли да нанижете у почетном оквиру након `set_index('Година')`.

In [8]:
import plotly.express as px

# Сортирали смо вредности по опадајућем редоследу да би CO2 био на дну (ascending=False)
graf = px.area(otisakTrendPlot.sort_values('Вредност у gha', ascending=False), 
                                            x="Година", y="Вредност у gha", 
                                            color="Тип", width=800,
                                            template='plotly_white')

graf.update_layout(margin=dict(l=50, r=20, t=20, b=50),
                   legend=dict(title='Компонента еколошког отиска', y=1, x=0.01, bgcolor='rgba(0,0,0,0)')
                  )

graf.update_xaxes(nticks=13)

graf.show()


Као што се види на основу графикона, глобални еколошки отисак је у сталном порасту у протеклих 50 година. Највећи пораст тиче се емисије гасова стаклене баште, тачније површина потребних да се они апсорбују. Можете показивачем миша прећи преко линија да бисте видели колике су биле вредности у појединачним годинама. Ипак, на основу графикона је тешко закључивати о тачним вредностима промена, па ћемо за крај израчунати и тачну разлику између крајње и почетне године за сваку компоненту. Употребићемо табелу са подацима коју смо употребили и за цртање графикона.

In [9]:
otisak1961 = otisakTrendPlot[otisakTrendPlot['Година']==1961][['Тип', 'Вредност у gha']]
otisak2017 = otisakTrendPlot[otisakTrendPlot['Година']==2017][['Тип', 'Вредност у gha']]
otisakOdnos = otisak1961.merge(otisak2017, on='Тип')
otisakOdnos

,Тип,Вредност у gha_x,Вредност у gha_y
0,Оранице,0.146385,0.322611
1,Пашњаци,0.085419,0.084762
2,Шуме,0.137855,0.170576
3,CO2,0.321651,1.060213
4,Риба,0.030571,0.056550
5,Земљиште,0.008470,0.039470


Најпре смо направили нове табеле са колонама *Тип* и *Вредност у gha* за 1961. и 2017. годину, а потом смо те две табеле спојили командом `merge()` повезујући редове по колони *Тип*. Остаје само да израчунамо разлику издвојених вредности.

In [10]:
otisakOdnos['Промена (%)'] = (((otisakOdnos['Вредност у gha_y'] - otisakOdnos['Вредност у gha_x']) / otisakOdnos['Вредност у gha_x']) * 100).round(2)
otisakOdnos[['Тип', 'Промена (%)']]

,Тип,Промена (%)
0,Оранице,2.20
1,Пашњаци,0.99
2,Шуме,1.24
3,CO2,3.30
4,Риба,1.85
5,Земљиште,4.66


Разлику између 2017. и 1961. године поделили смо вредношћу за 1961. како бисмо добили процентуални пораст. Неке разлике нису биле уочљиве на основу графикона, али сада су много јасније. На пример, потреба за површинама пашњака се у протеклих пола века чак и смањила, додуше незнатно. Исто тако, уочавамо да најзначајнији пораст није онај који се тиче емисије гасова већ онај који се односи на површине грађевинског земљишта. 

<div class="alert alert-block alert-info"><img src="slike/zad.png" align="left"/>Промене између година изразили смо процентуално што можда није најинтуитивнији начин. Како бисте изразили промену као умножак, тј. одговор на питање колико пута је отисак повећан? Покушајте да коригујете горњи кôд како бисте добили табелу са таквим вредностима.</div> 

<details>
<summary><img src="slike/click.png" align="right" width="70"/>
    <b>Прикажи одговор</b>
</summary>
Довољно је да поделите вредност за 2017. годину вредношћу за 1961. годину: <code>(otisakOdnos['Вредност у gha_y'] / otisakOdnos['Вредност у gha_x']).round(2)</code>.

**Како се мењао еколошки отисак у последњих пола века?**

Еколошки отисак константно расте од седамдесетих година 20. века. Иако је за овај раст добрим делом „заслужан“ пораст емисије угљен-диоксида, чини се да је озбиљнији проблем растућа потребa за грађевинским и обрадивим земљиштем. Ово друго очигледно подразумева крчење шума и уништавање биокапацитета.